In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
import pandas as pd
import time

## Financial reports

In [7]:
def init_driver():
    """Khởi tạo và trả về một trình duyệt Chrome WebDriver."""
    chrome_options = ChromeOptions()
    # chrome_options.add_argument("--headless=new")  # Headless mode mới
    # chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def scrape_financial_report(stock_code):
    """Lấy dữ liệu giao dịch từ trang web và gắn mã CK & tên công ty vào mỗi hàng."""
    driver = init_driver()
    wait = WebDriverWait(driver, 10)

    url = f'https://finance.vietstock.vn/{stock_code}/financials.htm?tab=BCTT'
    try:
        driver.get(url)

        wait = WebDriverWait(driver, 10)
        
        sidebar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-mobile-all"]/img')))
        sidebar.click()
        time.sleep(3)
        
        login_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="popup-menu-mobile"]/div/div[1]/div[2]/div/a[1]')))

        # Nhấp vào nút đăng nhập
        login_button.click()

        email_input = driver.find_element(By.ID, "txtEmailLogin")
        email_input.send_keys("dangdinhhung249@gmail.com")

        # Tìm và điền password
        password_input = driver.find_element(By.ID, "txtPassword")
        password_input.send_keys("money0918273645")
        
        # Nhấn Enter để đăng nhập
        password_input.send_keys(Keys.RETURN)

        # Chờ để kiểm tra kết quả đăng nhập
        time.sleep(3)
        
        select_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[1]/select[1]')))
        # Tạo đối tượng Select
        select = Select(select_element)

        # Chọn option "4 Period" theo value
        select.select_by_value("4")
        time.sleep(2)
        
        dropdown = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[1]/select[2]')))

        select__ = Select(dropdown)
        # Chọn "Year" theo giá trị (value)
        select__.select_by_value("NAM")
        time.sleep(2)
        
        select_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[1]/select[3]')))
        
        # Tạo đối tượng Select
        select_ = Select(select_element)

        # Chọn "Million Dong" theo giá trị value (1000000)
        select_.select_by_value("1000000")
        time.sleep(2)
        
        export = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dropdownMenuButton"]')))
        export.click()
        time.sleep(2)
        
        download = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[2]/div/div/a[2]')))
        download.click()
        time.sleep(3)
        
        while True:
            try:
                # Chờ nút có thể nhấn được (tối đa 5 giây mỗi lần)
                btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="finance-content"]/div/div/div[2]/div/div[1]/div[2]')))
                btn_class = btn.get_attribute("class")
                
                if "disabled" in btn_class:
                    break
                btn.click()
                time.sleep(3)
                
                export = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dropdownMenuButton"]')))
                export.click()
                time.sleep(2)
                
                
                download = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="finance-content"]/div/div/div[2]/div/div[2]/div[1]/div[2]/div/div/a[2]')))
                download.click()
                time.sleep(3)
            except:
                # Nếu nút không còn bấm được, thoát vòng lặp
                break
    finally:
        driver.quit()



In [28]:
df_stocks = pd.read_csv(r'/home/hamornic/FinalXXX/stock.csv')
df_stocks.head()

,Stock,Ngành,Tên công ty
0,VIC,Bất Động Sản,Tập đoàn VINGROUP - CTCP
1,VHM,Bất Động Sản,CTCP Vinhomes
2,BCM,Bất Động Sản,Tổng Công ty Đầu tư và Phát triển Công nghiệp ...
3,SSH,Bất Động Sản,CTCP Phát triển Sunshine Homes
4,VRE,Bất Động Sản,CTCP Vincom Retail


In [36]:
df=df_stocks.iloc[21:25]
df.head()

,Stock,Ngành,Tên công ty
21,CMT,Công Nghệ Thông tin,TCP Công nghệ Mạng và Truyền thông
22,PIA,Công Nghệ Thông tin,TCP Tin học Viễn thông Petrolimex
23,VTB,Công Nghệ Thông tin,TCP Viettronics Tân Bình
24,VCB,Ngân hàng,Ngân hàng TMCP Ngoại thương Việt Nam


In [26]:
df.tail()

,Stock,Ngành,Tên công ty
20,HPT,Công Nghệ Thông tin,TCP Dịch vụ Công nghệ Tin học HPT
21,CMT,Công Nghệ Thông tin,TCP Công nghệ Mạng và Truyền thông
22,PIA,Công Nghệ Thông tin,TCP Tin học Viễn thông Petrolimex
23,VTB,Công Nghệ Thông tin,TCP Viettronics Tân Bình
24,VCB,Ngân hàng,Ngân hàng TMCP Ngoại thương Việt Nam


In [37]:
for stock_code in df['Stock']:
    scrape_financial_report(stock_code)

In [28]:
scrape_financial_report('VCB')

[]

### Vốn hóa - Market cap

In [43]:
def init_driver():
    """Khởi tạo và trả về một trình duyệt Chrome WebDriver."""
    chrome_options = ChromeOptions()
    # chrome_options.add_argument("--headless=new")  # Headless mode mới
    # chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def scrape_financial_report(stock_code):
    """Lấy dữ liệu giao dịch từ trang web và gắn mã CK & tên công ty vào mỗi hàng."""
    driver = init_driver()
    wait = WebDriverWait(driver, 10)

    url = f'https://finance.vietstock.vn/{stock_code}/financials.htm?tab=BCTT'

    try:
        driver.get(url)
        print(f'Đang lấy vốn hóa của {stock_code}')
        # Chờ phần tử chứa Market Cap hiển thị
        market_cap_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="page-container"]/div[3]/div[1]/div[4]/div/div[2]/p[5]/b')))

        # Lấy giá trị Market Cap
        market_cap = market_cap_element.text.strip()

        return {"stock_code": stock_code, "market_cap": market_cap}

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu {stock_code}: {e}")
        return None
    finally:
        driver.quit()

In [40]:
df_stocks = pd.read_csv(r'/home/hamornic/FinalXXX/stock.csv')
df_stocks.head()

,Stock,Ngành,Tên công ty
0,VIC,Bất Động Sản,Tập đoàn VINGROUP - CTCP
1,VHM,Bất Động Sản,CTCP Vinhomes
2,BCM,Bất Động Sản,Tổng Công ty Đầu tư và Phát triển Công nghiệp ...
3,SSH,Bất Động Sản,CTCP Phát triển Sunshine Homes
4,VRE,Bất Động Sản,CTCP Vincom Retail


In [45]:
market_cap = []
for stock_code in df_stocks['Stock']:
    mc = scrape_financial_report(stock_code)
    market_cap.append(mc)

Đang lấy vốn hóa của VIC
Đang lấy vốn hóa của VHM
Đang lấy vốn hóa của BCM
Đang lấy vốn hóa của SSH
Đang lấy vốn hóa của VRE
Đang lấy vốn hóa của KDH
Đang lấy vốn hóa của KBC
Đang lấy vốn hóa của KSF
Đang lấy vốn hóa của BSR
Đang lấy vốn hóa của PVS
Đang lấy vốn hóa của PVD
Đang lấy vốn hóa của HGM
Đang lấy vốn hóa của MVB
Đang lấy vốn hóa của SQC
Lỗi khi lấy dữ liệu TMB: HTTPConnectionPool(host='localhost', port=53707): Read timed out. (read timeout=120)
Đang lấy vốn hóa của CST
Đang lấy vốn hóa của FPT
Đang lấy vốn hóa của CMG
Đang lấy vốn hóa của ITD
Đang lấy vốn hóa của POT
Đang lấy vốn hóa của HPT
Đang lấy vốn hóa của CMT
Đang lấy vốn hóa của PIA
Đang lấy vốn hóa của VTB
Đang lấy vốn hóa của VCB
Đang lấy vốn hóa của BID
Đang lấy vốn hóa của CTG
Đang lấy vốn hóa của TCB
Đang lấy vốn hóa của VPB
Đang lấy vốn hóa của MBB
Đang lấy vốn hóa của ACB
Đang lấy vốn hóa của LPB


In [46]:
mc = scrape_financial_report('TMB')
market_cap.append(mc)

Đang lấy vốn hóa của TMB


In [48]:
clean_data = [d for d in market_cap if d is not None]

In [49]:
df = pd.DataFrame(clean_data)
df.head()

,stock_code,market_cap
0,VIC,"202,654.06"
1,VHM,"198,182.63"
2,BCM,"81,454.50"
3,SSH,"37,800"
4,VRE,"41,924.27"


In [50]:
df.to_csv("/home/hamornic/FinalXXX/Financial Report Processed/market_cap.csv", index=False, encoding="utf-8")

## Enterprises

### Lấy link của tất cả các stock có section là viễn thông.

In [38]:
# Khởi tạo trình duyệt
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Mở trang web
url = "https://finance.vietstock.vn/trang-thai-co-phieu/danh-sach-niem-yet?page=1"

try:
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    driver.execute_script("window.stop();")

    next_button = driver.find_element(By.XPATH, '//*[@id="btn-page-next"]')
    next_button.click()
    
    #Login
    # Tìm và điền email
    email_input = driver.find_element(By.ID, "txtEmailLogin")
    email_input.send_keys("dangdinhhung249@gmail.com")

    # Tìm và điền password
    password_input = driver.find_element(By.ID, "txtPassword")
    password_input.send_keys("money0918273645")
    
    # Nhấn Enter để đăng nhập
    password_input.send_keys(Keys.RETURN)

    # Chờ để kiểm tra kết quả đăng nhập
    time.sleep(3)

    

    print(1)
    # Tìm phần tử select và chọn "Communication Services"
    dropdown = driver.find_element(By.XPATH, '//*[@id="listing-az"]/div/div[1]/div[1]/div[1]/select')
    print(10)
    select = Select(dropdown)
    select.select_by_value("50")  # Giá trị "50" tương ứng với Communication Services

    # Nhấn nút "Search"
    search_button = driver.find_element(By.XPATH, '//*[@id="listing-az"]/div/div[1]/div[1]/button')
    search_button.click()
    # Chờ trang load dữ liệu mới    
    time.sleep(5)
    
    # Tìm phần tử select
    select_element = driver.find_element("name", "pageSize")

    # Tạo đối tượng Select
    select = Select(select_element)

    # Chọn giá trị 50
    select.select_by_value("50")
    time.sleep(2)
    
    
    # Tìm tất cả các hàng trong bảng (bỏ qua hàng tiêu đề)
    rows = driver.find_elements(By.XPATH, "//table/tbody/tr")

    # Duyệt qua từng hàng và lấy dữ liệu từng ô
    data = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        row_data = [col.text for col in cols]

        # Tìm thẻ <a> trong hàng để lấy link
        try:
            link = row.find_element(By.CSS_SELECTOR, "a.title-link").get_attribute("href")
        except:
            link = ""  # Nếu không tìm thấy link, để trống

        row_data.append(link)  # Thêm link vào hàng dữ liệu
        data.append(row_data)

    driver.quit()
    # Đóng trình duyệt sau khi hoàn thành (có thể bỏ nếu cần thao tác tiếp)
except:
    driver.quit()

In [3]:
data

[['', '', ''],
 ['', '', ''],
 ['1',
  'ABC',
  'CTCP Truyền thông VMG',
  'Dịch vụ viễn thông',
  'Dịch vụ viễn thông',
  'Dịch vụ viễn thông đa dạng',
  'UPCoM',
  '18/08/2016',
  '20,393,000',
  'https://finance.vietstock.vn/ABC-ctcp-truyen-thong-vmg.htm'],
 ['2',
  'ABR',
  'CTCP Đầu tư Nhãn hiệu Việt',
  'Dịch vụ viễn thông',
  'Dịch vụ viễn thông',
  'Dịch vụ viễn thông đa dạng',
  'HOSE',
  '03/03/2022',
  '20,000,000',
  'https://finance.vietstock.vn/ABR-ctcp-dau-tu-nhan-hieu-viet.htm'],
 ['3',
  'ADC',
  'CTCP Mỹ thuật và Truyền thông',
  'Dịch vụ viễn thông',
  'Truyền thông và giải trí',
  'Truyền thông',
  'HNX',
  '24/12/2010',
  '3,977,936',
  'https://finance.vietstock.vn/ADC-ctcp-my-thuat-va-truyen-thong.htm'],
 ['4',
  'ADG',
  'CTCP Clever Group',
  'Dịch vụ viễn thông',
  'Truyền thông và giải trí',
  'Truyền thông',
  'HOSE',
  '22/02/2021',
  '21,380,521',
  'https://finance.vietstock.vn/ADG-ctcp-clever-group.htm'],
 ['5',
  'BDB',
  'CTCP Sách và Thiết bị Bình Địn

In [4]:
df = pd.DataFrame(data, columns=["STT", "Mã CK", "Tên công ty", "Ngành cấp 1", "Ngành cấp 2", 
                                 "Ngành cấp 3", "Sàn", "Ngày GDĐT", "Khối lượng NY/ĐKGD", "Link_detail"])
df.head()

,STT,Mã CK,Tên công ty,Ngành cấp 1,Ngành cấp 2,Ngành cấp 3,Sàn,Ngày GDĐT,Khối lượng NY/ĐKGD,Link_detail
0,,,,None,None,None,None,None,None,None
1,,,,None,None,None,None,None,None,None
2,1,ABC,CTCP Truyền thông VMG,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,UPCoM,18/08/2016,"20,393,000",https://finance.vietstock.vn/ABC-ctcp-truyen-t...
3,2,ABR,CTCP Đầu tư Nhãn hiệu Việt,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,HOSE,03/03/2022,"20,000,000",https://finance.vietstock.vn/ABR-ctcp-dau-tu-n...
4,3,ADC,CTCP Mỹ thuật và Truyền thông,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,24/12/2010,"3,977,936",https://finance.vietstock.vn/ADC-ctcp-my-thuat...


In [5]:
df.dropna(inplace=True)

In [6]:
df.reset_index(drop=True, inplace=True)

df.head()

,STT,Mã CK,Tên công ty,Ngành cấp 1,Ngành cấp 2,Ngành cấp 3,Sàn,Ngày GDĐT,Khối lượng NY/ĐKGD,Link_detail
0,1,ABC,CTCP Truyền thông VMG,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,UPCoM,18/08/2016,"20,393,000",https://finance.vietstock.vn/ABC-ctcp-truyen-t...
1,2,ABR,CTCP Đầu tư Nhãn hiệu Việt,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,HOSE,03/03/2022,"20,000,000",https://finance.vietstock.vn/ABR-ctcp-dau-tu-n...
2,3,ADC,CTCP Mỹ thuật và Truyền thông,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,24/12/2010,"3,977,936",https://finance.vietstock.vn/ADC-ctcp-my-thuat...
3,4,ADG,CTCP Clever Group,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HOSE,22/02/2021,"21,380,521",https://finance.vietstock.vn/ADG-ctcp-clever-g...
4,5,BDB,CTCP Sách và Thiết bị Bình Định,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,29/01/2010,"1,126,474",https://finance.vietstock.vn/BDB-ctcp-sach-va-...


In [56]:
df.to_csv(r'/home/hamornic/FinalXXX/DataScraped/enterprise/all_link_stock.csv')

### Lấy thông tin chi tiết từng link một.

In [33]:
# Cấu hình Chrome
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
chrome_options.add_argument("--disable-css")  # Tắt CSS
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Khởi tạo trình duyệt
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Danh sách để lưu kết quả
all_results = []

def scrape_company_info(url):
    """Hàm lấy thông tin công ty từ URL và lưu vào danh sách all_results"""
    try:
        driver.get(url)
        time.sleep(2)  # Đợi trang tải
        
        # Lấy thông tin từ Profile 1
        name = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[2]').text
        abbreviation = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[3]').text
        address = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[5]').text.replace("Address: ", "")
        telephone = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[7]').text.replace("Telephone: ", "")
        fax = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[8]').text.replace("Fax: ", "")
        email = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[9]').text
        website = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[10]').text

        # Click vào tab "Listing"
        driver.find_element(By.XPATH, '//*[@id="view-content"]/div[2]/div[2]/ul/li[2]/a').click()
        time.sleep(2)

        exchange = driver.find_element(By.XPATH, '//*[@id="profile-2"]/p[1]').text.replace("Exchange: ", "")
        listing_date = driver.find_element(By.XPATH, '//*[@id="profile-2"]/p[4]').text.replace("Listing date: ", "")

        # Click vào tab "Tax Code"
        driver.find_element(By.XPATH, '//*[@id="view-content"]/div[2]/div[2]/ul/li[3]/a').click()
        time.sleep(2)

        tax_code = driver.find_element(By.XPATH, '//*[@id="profile-3"]/p[2]').text.replace("Tax code: ", "")

        print(url, '----------------')
        print(f"Company Name: {name}")
        print(f"Abbreviation: {abbreviation}")
        print(f"Address: {address}")
        print(f"Telephone: {telephone}")
        print(f"Fax: {fax}")
        print(f"Email: {email}")
        print(f"Website: {website}")
        print(f"Exchange: {exchange}")
        print(f"Listing Date: {listing_date}")
        print(f"Tax Code: {tax_code}")
        # Tạo dictionary kết quả
        company_info = {
            "Company Name": name,
            "Abbreviation": abbreviation,
            "Address": address,
            "Telephone": telephone,
            "Fax": fax,
            "Email": email,
            "Website": website,
            "Tax Code": tax_code,
            "Exchange": exchange,
            "Listing Date": listing_date
        }
        
        # Thêm vào danh sách tổng hợp
        all_results.append(company_info)
        
        #driver.quit() 
    
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ {url}: {e}")

In [34]:
# Duyệt từng URL và gọi hàm
for url in df["Link_detail"]:
    scrape_company_info(url)



https://finance.vietstock.vn/ABC-ctcp-truyen-thong-vmg.htm ----------------
Company Name: Tên đầy đủ : CTCP Truyền thông VMG
Abbreviation: Tên tiếng Anh : VMG Media JSC
Address: Địa chỉ : Tầng 6 tòa nhà PEAKVIEW Tower Số 36 Hoàng Cầu - P. Ô Chợ Dừa - Q. Đông Đa - Tp. Hà Nội
Telephone: Điện thoại : (84.24) 3537 8820
Fax: Fax : (84.24) 3772 6091
Email: Email :hoangnam.nguyen@vmgmedia.vn
Website: Website :https://vmgmedia.vn
Exchange: Sàn giao dịch : UPCoM
Listing Date: Ngày niêm yết : 18/08/2016
Tax Code: Mã số thuế : 0101883619
https://finance.vietstock.vn/ABR-ctcp-dau-tu-nhan-hieu-viet.htm ----------------
Company Name: Tên đầy đủ : CTCP Đầu tư Nhãn hiệu Việt
Abbreviation: Tên tiếng Anh : Viet Brand Invest JSC
Address: Địa chỉ : Tầng 4 - Tòa nhà Broadway E - Số 2 Bertrand Russell - P. Tân Phú - Q.7 - Tp. HCM
Telephone: Điện thoại : (84.28) 5413 8366
Fax: Fax : (84.28) 5413 8370
Email: Email :info@vbinvest.com.vn
Website: Website :http://vbinvest.com.vn
Exchange: Sàn giao dịch : HOSE
Li

In [35]:
df['Link_detail'].shape

(45,)

In [36]:
len(all_results)

44

Thiếu 1 là link này: https://finance.vietstock.vn/ECI-ctcp-tap-doan-eci.htm (có thể nhìn từ output ở hàm scrape_company_info)
Lý do là trong quá trình chạy chẳng may dính quảng cáo che mất nên k click được.

In [37]:
#Gọi lại để lấy nốt 1 cái.
scrape_company_info('https://finance.vietstock.vn/ECI-ctcp-tap-doan-eci.htm')

https://finance.vietstock.vn/ECI-ctcp-tap-doan-eci.htm ----------------
Company Name: Tên đầy đủ : CTCP Tập Đoàn ECI
Abbreviation: Tên tiếng Anh : ECI Group Joint Stock Company
Address: Địa chỉ : Số 45 Hàng Chuối- P. Phạm Đình Hổ - Q.Hai Bà Trưng - Tp.Hà Nội
Telephone: Điện thoại : (84.24) 3971 6096
Fax: Fax : (84.24) 3971 8847
Email: Email :info@ecigroup.com.vn
Website: Website :https://ecigroup.com.vn/
Exchange: Sàn giao dịch : HNX
Listing Date: Ngày niêm yết : 16/06/2009
Tax Code: Mã số thuế : 0102137109


In [38]:
len(all_results)

45

In [44]:
# Chuyển kết quả thành DataFrame
df_results = pd.DataFrame(all_results)

df_results.head()

,Company Name,Abbreviation,Address,Telephone,Fax,Email,Website,Tax Code,Exchange,Listing Date
0,Tên đầy đủ : CTCP Truyền thông VMG,Tên tiếng Anh : VMG Media JSC,Địa chỉ : Tầng 6 tòa nhà PEAKVIEW Tower Số 36 ...,Điện thoại : (84.24) 3537 8820,Fax : (84.24) 3772 6091,Email :hoangnam.nguyen@vmgmedia.vn,Website :https://vmgmedia.vn,Mã số thuế : 0101883619,Sàn giao dịch : UPCoM,Ngày niêm yết : 18/08/2016
1,Tên đầy đủ : CTCP Đầu tư Nhãn hiệu Việt,Tên tiếng Anh : Viet Brand Invest JSC,Địa chỉ : Tầng 4 - Tòa nhà Broadway E - Số 2 B...,Điện thoại : (84.28) 5413 8366,Fax : (84.28) 5413 8370,Email :info@vbinvest.com.vn,Website :http://vbinvest.com.vn,Mã số thuế : 0312080709,Sàn giao dịch : HOSE,Ngày niêm yết : 03/03/2022
2,Tên đầy đủ : CTCP Mỹ thuật và Truyền thông,Tên tiếng Anh : Art Design And Communication JSC,Địa chỉ : Tầng 12A - Tòa nhà Diamond Flower - ...,Điện thoại : (84.24) 3512 2163,Fax : (84.24) 3512 1385,Email :webmaster@adc.net.vn,Website :http://adc.net.vn,Mã số thuế : 0102365521,Sàn giao dịch : HNX,Ngày niêm yết : 24/12/2010
3,Tên đầy đủ : CTCP Clever Group,Tên tiếng Anh : Clever Group Corporation,Địa chỉ : Tầng 3 Nhà G1 tòa Five Star - Số 2 K...,Điện thoại : (84.247) 3046 066,Fax :,Email :contact@cleverads.vn,Website :https://clevergroup.vn,Mã số thuế : 0102850165,Sàn giao dịch : HOSE,Ngày niêm yết : 22/02/2021
4,Tên đầy đủ : CTCP Sách và Thiết bị Bình Định,Tên tiếng Anh : Binh Dinh Book & Equipment Joi...,Địa chỉ : Số 219 Nguyễn Lữ - Tp.Quy Nhơn - T.B...,Điện thoại : (84.256) 352 2645,Fax : (84.256) 352 2853,Email :bidisabico@vnn.vn,Website :http://www.sachthietbibinhdinh.com.vn,Mã số thuế : 4100259437,Sàn giao dịch : HNX,Ngày niêm yết : 29/01/2010


In [54]:
df.head()

,STT,Mã CK,Tên công ty,Ngành cấp 1,Ngành cấp 2,Ngành cấp 3,Sàn,Ngày GDĐT,Khối lượng NY/ĐKGD,Link_detail
0,1,ABC,CTCP Truyền thông VMG,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,UPCoM,18/08/2016,"20,393,000",https://finance.vietstock.vn/ABC-ctcp-truyen-t...
1,2,ABR,CTCP Đầu tư Nhãn hiệu Việt,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,HOSE,03/03/2022,"20,000,000",https://finance.vietstock.vn/ABR-ctcp-dau-tu-n...
2,3,ADC,CTCP Mỹ thuật và Truyền thông,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,24/12/2010,"3,977,936",https://finance.vietstock.vn/ADC-ctcp-my-thuat...
3,4,ADG,CTCP Clever Group,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HOSE,22/02/2021,"21,380,521",https://finance.vietstock.vn/ADG-ctcp-clever-g...
4,5,BDB,CTCP Sách và Thiết bị Bình Định,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,29/01/2010,"1,126,474",https://finance.vietstock.vn/BDB-ctcp-sach-va-...


Có vẻ khớp. :))

In [52]:
df_results['stock_code'] = df['Mã CK']

In [53]:
df_results.head()

,Company Name,Abbreviation,Address,Telephone,Fax,Email,Website,Tax Code,Exchange,Listing Date,stock_code
0,Tên đầy đủ : CTCP Truyền thông VMG,Tên tiếng Anh : VMG Media JSC,Địa chỉ : Tầng 6 tòa nhà PEAKVIEW Tower Số 36 ...,Điện thoại : (84.24) 3537 8820,Fax : (84.24) 3772 6091,Email :hoangnam.nguyen@vmgmedia.vn,Website :https://vmgmedia.vn,Mã số thuế : 0101883619,Sàn giao dịch : UPCoM,Ngày niêm yết : 18/08/2016,ABC
1,Tên đầy đủ : CTCP Đầu tư Nhãn hiệu Việt,Tên tiếng Anh : Viet Brand Invest JSC,Địa chỉ : Tầng 4 - Tòa nhà Broadway E - Số 2 B...,Điện thoại : (84.28) 5413 8366,Fax : (84.28) 5413 8370,Email :info@vbinvest.com.vn,Website :http://vbinvest.com.vn,Mã số thuế : 0312080709,Sàn giao dịch : HOSE,Ngày niêm yết : 03/03/2022,ABR
2,Tên đầy đủ : CTCP Mỹ thuật và Truyền thông,Tên tiếng Anh : Art Design And Communication JSC,Địa chỉ : Tầng 12A - Tòa nhà Diamond Flower - ...,Điện thoại : (84.24) 3512 2163,Fax : (84.24) 3512 1385,Email :webmaster@adc.net.vn,Website :http://adc.net.vn,Mã số thuế : 0102365521,Sàn giao dịch : HNX,Ngày niêm yết : 24/12/2010,ADC
3,Tên đầy đủ : CTCP Clever Group,Tên tiếng Anh : Clever Group Corporation,Địa chỉ : Tầng 3 Nhà G1 tòa Five Star - Số 2 K...,Điện thoại : (84.247) 3046 066,Fax :,Email :contact@cleverads.vn,Website :https://clevergroup.vn,Mã số thuế : 0102850165,Sàn giao dịch : HOSE,Ngày niêm yết : 22/02/2021,ADG
4,Tên đầy đủ : CTCP Sách và Thiết bị Bình Định,Tên tiếng Anh : Binh Dinh Book & Equipment Joi...,Địa chỉ : Số 219 Nguyễn Lữ - Tp.Quy Nhơn - T.B...,Điện thoại : (84.256) 352 2645,Fax : (84.256) 352 2853,Email :bidisabico@vnn.vn,Website :http://www.sachthietbibinhdinh.com.vn,Mã số thuế : 4100259437,Sàn giao dịch : HNX,Ngày niêm yết : 29/01/2010,BDB


In [55]:
#Lưu thành dataframe
df_results.to_csv(r'/home/hamornic/FinalXXX/DataScraped/enterprise/enterprise.csv')

----------------

### Nháp

In [22]:
# Cấu hình Chrome
chrome_options = ChromeOptions()
# chrome_options.add_argument("--headless")  # Chạy chế độ không hiển thị giao diện

# Khởi tạo trình duyệt
service = Service(ChromeDriverManager().install())
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
chrome_options.add_argument("--disable-css")  # Tắt CSS
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=service, options=chrome_options)

# Truy cập trang web
url = "https://finance.vietstock.vn/ADG-ctcp-clever-group.htm"
driver.get(url)
# wait = WebDriverWait(driver, 10)

# Lấy thông tin từ Profile 1
name = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[2]').text
abbreviation = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[3]').text
address = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[5]').text.replace("Address: ", "")
telephone = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[7]').text.replace("Telephone: ", "")
fax = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[8]').text.replace("Fax: ", "")
email = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[9]').text
website = driver.find_element(By.XPATH, '//*[@id="profile-1"]/p[10]').text

element = driver.find_element(By.XPATH, '//*[@id="view-content"]/div[2]/div[2]/ul/li[2]/a')
element.click()
time.sleep(2)

exchange = driver.find_element(By.XPATH, '//*[@id="profile-2"]/p[1]').text.replace("Exchange: ", "")
listing_date = driver.find_element(By.XPATH, '//*[@id="profile-2"]/p[4]').text.replace("Listing date: ", "")

element = driver.find_element(By.XPATH, '//*[@id="view-content"]/div[2]/div[2]/ul/li[3]/a')
element.click()
time.sleep(2)

tax_code = driver.find_element(By.XPATH, '//*[@id="profile-3"]/p[2]').text.replace("Tax code: ", "")


# In thông tin ra màn hình
print("Company Name:", name)
print("Abbreviation:", abbreviation)
print("Address:", address)
print("Telephone:", telephone)
print("Fax:", fax)
print("Email:", email)
print("Website:", website)
print(f"Tax Code: {tax_code}")
print(f"Exchange: {exchange}")
print(f"Listing Date: {listing_date}")

# Đóng trình duyệt
# driver.quit()


Company Name: Tên đầy đủ : CTCP Clever Group
Abbreviation: Tên tiếng Anh : Clever Group Corporation
Address: Địa chỉ : Tầng 3 Nhà G1 tòa Five Star - Số 2 Kim Giang - P. Kim Giang - Q. Thanh Xuân - Tp. Hà Nội
Telephone: Điện thoại : (84.247) 3046 066
Fax: Fax :
Email: Email :contact@cleverads.vn
Website: Website :https://clevergroup.vn
Tax Code: Mã số thuế : 0102850165
Exchange: Sàn giao dịch : HOSE
Listing Date: Ngày niêm yết : 22/02/2021


## Transaction History

In [23]:

def init_driver():
    """Khởi tạo và trả về một trình duyệt Chrome WebDriver."""
    chrome_options = ChromeOptions()
    chrome_options.add_argument("--headless=new")  # Headless mode mới
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def scrape_table_data(stock_code, company_name, url):
    """Lấy dữ liệu giao dịch từ trang web và gắn mã CK & tên công ty vào mỗi hàng."""
    driver = init_driver()
    wait = WebDriverWait(driver, 10)
    data = []

    try:
        driver.get(url)

        # Nhập mã chứng khoán
        input_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ContentPlaceHolder1_ctl00_acp_inp_disclosure"]')))
        input_box.clear()
        time.sleep(2)
        input_box.send_keys(stock_code)
        time.sleep(2)

        # input_box.send_keys(Keys.RETURN) 
        # Nhấn vào nút tìm kiếm
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="owner-find"]')))
        search_button.click()
        time.sleep(3) 

        print(f'Nhập thành công: {stock_code} - {company_name}')

        # Chờ bảng dữ liệu xuất hiện
        wait.until(EC.presence_of_element_located((By.ID, "owner-contents-table")))

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'pagination-item')]/p")))
        # Xác định tổng số trang
        pagination_items = driver.find_elements(By.XPATH, "//div[contains(@class, 'pagination-item')]/p")
        page_numbers = [int(item.get_attribute("title")) for item in pagination_items if item.get_attribute("title") and item.get_attribute("title").isdigit()]
        last_page = max(page_numbers) if page_numbers else 1
    

        print(f"Tổng số trang: {last_page}")

        for page in range(1, last_page + 1):
            WebDriverWait(driver, 50)
            print(f"Đang lấy dữ liệu trang {page}...")

            # Chuyển trang nếu cần
            if page > 1:
                # page_xpath = f'//*[@id="divStart"]/div/div[3]/div[3]'
                page_xpath = f"//div[contains(@class, 'pagination-item')]/p[@title='{page}']"
                try:
                    page_button = wait.until(EC.element_to_be_clickable((By.XPATH, page_xpath)))
                    driver.execute_script("arguments[0].click();", page_button)  # Click bằng JavaScript để tránh lỗi

                    wait.until(EC.staleness_of(page_button))  # Đợi trang tải lại
                except Exception as e:
                    print(f"Lỗi khi chuyển sang trang {page}: {e}")
                    continue  # Bỏ qua nếu không thể chuyển trang

            # Lấy dữ liệu từ bảng
            try:
                table = wait.until(EC.presence_of_element_located((By.ID, "owner-contents-table")))
                rows = table.find_elements(By.TAG_NAME, "tr")

                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    row_data = [col.text.strip() for col in cols]
                    if row_data:
                        row_data.insert(0, stock_code)    # Thêm mã CK vào đầu
                        row_data.insert(1, company_name)  # Thêm tên công ty vào sau mã CK
                        data.append(row_data)

            except Exception as e:
                print(f"Lỗi khi lấy dữ liệu từ trang {page}: {e}")

    finally:
        driver.quit()
    
    return data



In [20]:
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-cmg-1.chn#data"
cost_history = []  # Danh sách để lưu toàn bộ dữ liệu

data_ = scrape_table_data('KDH', 'CTCP Đầu tư và Kinh doanh Nhà Khang Điền', url)
cost_history.extend(data_) 

Nhập thành công: KDH - CTCP Đầu tư và Kinh doanh Nhà Khang Điền
Tổng số trang: 189
Đang lấy dữ liệu trang 71...
Đang lấy dữ liệu trang 72...
Đang lấy dữ liệu trang 73...
Đang lấy dữ liệu trang 74...


##### Crawl dữ liệu của các ngành khác.

In [7]:
df = pd.read_csv(r'/home/hamornic/FinalXXX/stock.csv')
df.head()

,Stock,Ngành,Tên công ty
0,VIC,Bất Động Sản,Tập đoàn VINGROUP - CTCP
1,VHM,Bất Động Sản,CTCP Vinhomes
2,BCM,Bất Động Sản,Tổng Công ty Đầu tư và Phát triển Công nghiệp ...
3,SSH,Bất Động Sản,CTCP Phát triển Sunshine Homes
4,VRE,Bất Động Sản,CTCP Vincom Retail


In [8]:
df = df.iloc[15:]
df

,Stock,Ngành,Tên công ty
15,CST,Năng Lượng,CTCP Than Cao Sơn - TKV
16,FPT,Công Nghệ Thông tin,TCP FPT
17,CMG,Công Nghệ Thông tin,TCP Tập đoàn Công nghệ CMC
18,ITD,Công Nghệ Thông tin,TCP Công nghệ Tiên Phong
19,POT,Công Nghệ Thông tin,TCP Thiết bị Bưu điện
20,HPT,Công Nghệ Thông tin,TCP Dịch vụ Công nghệ Tin học HPT
21,CMT,Công Nghệ Thông tin,TCP Công nghệ Mạng và Truyền thông
22,PIA,Công Nghệ Thông tin,TCP Tin học Viễn thông Petrolimex
23,VTB,Công Nghệ Thông tin,TCP Viettronics Tân Bình
24,VCB,Ngân hàng,Ngân hàng TMCP Ngoại thương Việt Nam


In [9]:
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-cmg-1.chn#data"

cost_history = []  # Danh sách để lưu toàn bộ dữ liệu

for index, row in df.iterrows():
    stock_code = row['Stock']
    company_name = row['Tên công ty']
    
    data_ = scrape_table_data(stock_code, company_name, url)
    cost_history.extend(data_)  # Thêm dữ liệu vào danh sách chính

Nhập thành công: CST - CTCP Than Cao Sơn - TKV
Tổng số trang: 2
Đang lấy dữ liệu trang 1...
Đang lấy dữ liệu trang 2...
Nhập thành công: FPT - TCP FPT
Tổng số trang: 228
Đang lấy dữ liệu trang 1...
Đang lấy dữ liệu trang 2...
Đang lấy dữ liệu trang 3...
Đang lấy dữ liệu trang 4...
Đang lấy dữ liệu trang 5...
Đang lấy dữ liệu trang 6...
Đang lấy dữ liệu trang 7...
Đang lấy dữ liệu trang 8...
Đang lấy dữ liệu trang 9...
Đang lấy dữ liệu trang 10...
Đang lấy dữ liệu trang 11...
Đang lấy dữ liệu trang 12...
Đang lấy dữ liệu trang 13...
Đang lấy dữ liệu trang 14...
Đang lấy dữ liệu trang 15...
Đang lấy dữ liệu trang 16...
Đang lấy dữ liệu trang 17...
Đang lấy dữ liệu trang 18...
Đang lấy dữ liệu trang 19...
Đang lấy dữ liệu trang 20...
Đang lấy dữ liệu trang 21...
Đang lấy dữ liệu trang 22...
Đang lấy dữ liệu trang 23...
Đang lấy dữ liệu trang 24...
Đang lấy dữ liệu trang 25...
Đang lấy dữ liệu trang 26...
Đang lấy dữ liệu trang 27...
Đang lấy dữ liệu trang 28...
Đang lấy dữ liệu trang 29..

##### Crawl viễn thôgn

In [60]:
df.head()

,STT,Mã CK,Tên công ty,Ngành cấp 1,Ngành cấp 2,Ngành cấp 3,Sàn,Ngày GDĐT,Khối lượng NY/ĐKGD,Link_detail
0,1,ABC,CTCP Truyền thông VMG,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,UPCoM,18/08/2016,"20,393,000",https://finance.vietstock.vn/ABC-ctcp-truyen-t...
1,2,ABR,CTCP Đầu tư Nhãn hiệu Việt,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,HOSE,03/03/2022,"20,000,000",https://finance.vietstock.vn/ABR-ctcp-dau-tu-n...
2,3,ADC,CTCP Mỹ thuật và Truyền thông,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,24/12/2010,"3,977,936",https://finance.vietstock.vn/ADC-ctcp-my-thuat...
3,4,ADG,CTCP Clever Group,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HOSE,22/02/2021,"21,380,521",https://finance.vietstock.vn/ADG-ctcp-clever-g...
4,5,BDB,CTCP Sách và Thiết bị Bình Định,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,29/01/2010,"1,126,474",https://finance.vietstock.vn/BDB-ctcp-sach-va-...


In [51]:
# def scrape_table_data(ma_ck, url):
#     """Lấy dữ liệu giao dịch từ trang web."""
#     driver = init_driver()
#     wait = WebDriverWait(driver, 10)
#     data = []
#     try:
#         driver.get(url)
#         wait = WebDriverWait(driver, 10)
        
#         input_box = driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_ctl00_acp_inp_disclosure"]')
        
#         # Xóa nội dung cũ (nếu có)
#         input_box.clear()
#         time.sleep(1)
        
#         # Nhập mã chứng khoán
#         input_box.send_keys(ma_ck)
#         time.sleep(1)
        
#         # Tìm và nhấn vào nút xem
#         search_button = driver.find_element(By.XPATH, '//*[@id="owner-find"]')
#         search_button.click()
#         time.sleep(3)  # Đợi kết quả hiển thị
        
#         print(f'Nhập thành công: {ma_ck}')

#         # Chờ pagination xuất hiện
#         wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'pagination-item')]/p")))
        

#         # Tìm danh sách số trang (loại bỏ ...)
#         pagination_items = driver.find_elements(By.XPATH, "//div[contains(@class, 'pagination-item')]/p")
#         page_numbers = [int(item.get_attribute("title")) for item in pagination_items if item.get_attribute("title") and item.get_attribute("title").isdigit()]
#         last_page = max(page_numbers) if page_numbers else 1
        
#         print(f"Tổng số trang: {last_page}")

#         for page in range(1, last_page + 1):
#             print(f"Đang lấy dữ liệu trang {page}...")

#             # Click vào số trang
#             page_xpath = f"//div[contains(@class, 'pagination-item')]/p[@title='{page}']"

#             try:
#                 page_button = wait.until(EC.element_to_be_clickable((By.XPATH, page_xpath)))
#                 driver.execute_script("arguments[0].click();", page_button)
#             except Exception as e:
#                 print(f"Lỗi khi click trang {page}: {e}")
#                 continue  # Bỏ qua nếu không thể click
            
#             # Chờ dữ liệu trong bảng tải lại
#             time.sleep(2)  # Hoặc dùng WebDriverWait nếu có điều kiện phù hợp

#             # Lấy dữ liệu từ bảng sau khi trang mới đã tải xong
#             try:
#                 table = wait.until(EC.presence_of_element_located((By.ID, "owner-contents-table")))
#                 rows = table.find_elements(By.TAG_NAME, "tr")

#                 for row in rows:
#                     try:
#                         cols = row.find_elements(By.TAG_NAME, "td")
#                         row_data = [col.text.strip() for col in cols]
#                         if row_data:
#                             data.append(row_data)
#                     except Exception as e:
#                         print(f"Lỗi khi lấy dữ liệu từ hàng: {e}")

#             except Exception as e:
#                 print(f"Lỗi khi lấy bảng dữ liệu trang {page}: {e}")

#     finally:
#         driver.quit()
    
#     return data

In [27]:
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-cmg-4.chn#data"

cost_history = []  # Danh sách để lưu toàn bộ dữ liệu

for index, row in df.iterrows():
    stock_code = row['Mã CK']
    company_name = row['Tên công ty']
    
    data_ = scrape_table_data(stock_code, company_name, url)
    cost_history.extend(data_)  # Thêm dữ liệu vào danh sách chính

Nhập thành công: ABC - CTCP Truyền thông VMG
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: ABR - CTCP Đầu tư Nhãn hiệu Việt
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: ADC - CTCP Mỹ thuật và Truyền thông
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: ADG - CTCP Clever Group
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: BDB - CTCP Sách và Thiết bị Bình Định
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: BED - CTCP Sách và Thiết bị Trường học Đà Nẵng
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: BST - CTCP Sách và Thiết bị Bình Thuận
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: CKV - CTCP COKYVINA
Tổng số trang: 1
Đang lấy dữ liệu trang 1...
Nhập thành công: CTR - Tổng Công ty cổ phần Công trình Viettel
Tổng số trang: 21
Đang lấy dữ liệu trang 1...
Đang lấy dữ liệu trang 2...
Đang lấy dữ liệu trang 3...
Đang lấy dữ liệu trang 4...
Đang lấy dữ liệu trang 5...
Đang lấy dữ liệu t

In [11]:
df_ = df.iloc[34:]

In [12]:
df_

,STT,Mã CK,Tên công ty,Ngành cấp 1,Ngành cấp 2,Ngành cấp 3,Sàn,Ngày GDĐT,Khối lượng NY/ĐKGD,Link_detail
34,35,STC,CTCP Sách và Thiết bị Trường học Thành phố Hồ ...,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,27/12/2006,"5,665,530",https://finance.vietstock.vn/STC-ctcp-sach-va-...
35,36,TPH,CTCP In sách giáo khoa tại Thành phố Hà Nội,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,HNX,15/12/2006,"2,095,985",https://finance.vietstock.vn/TPH-ctcp-in-sach-...
36,37,TTN,CTCP Công nghệ và Truyền thông Việt Nam,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,UPCoM,19/04/2017,"36,727,500",https://finance.vietstock.vn/TTN-ctcp-cong-ngh...
37,38,VGI,Tổng Công ty cổ phần Đầu tư Quốc tế Viettel,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông không dây,UPCoM,25/09/2018,"3,043,811,200",https://finance.vietstock.vn/VGI-tong-cong-ty-...
38,39,VIE,CTCP Công nghệ Viễn thông VITECO,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,UPCoM,03/08/2022,"5,061,244",https://finance.vietstock.vn/VIE-ctcp-cong-ngh...
39,40,VNB,CTCP Sách Việt Nam,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,UPCoM,15/07/2016,"67,909,960",https://finance.vietstock.vn/VNB-ctcp-sach-vie...
40,41,VNX,CTCP Quảng cáo và Hội chợ Thương mại,Dịch vụ viễn thông,Truyền thông và giải trí,Truyền thông,UPCoM,10/06/2010,"1,224,948",https://finance.vietstock.vn/VNX-ctcp-quang-ca...
41,42,VNZ,CTCP VNG,Dịch vụ viễn thông,Truyền thông và giải trí,Giải trí,UPCoM,05/01/2023,"28,736,000",https://finance.vietstock.vn/VNZ-ctcp-vng.htm
42,43,VTC,CTCP Viễn thông VTC,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông đa dạng,HNX,12/02/2003,"4,534,696",https://finance.vietstock.vn/VTC-ctcp-vien-tho...
43,44,VTK,CTCP Tư Vấn Và Dịch Vụ Viettel,Dịch vụ viễn thông,Dịch vụ viễn thông,Dịch vụ viễn thông không dây,UPCoM,18/12/2018,"9,383,230",https://finance.vietstock.vn/VTK-ctcp-tu-van-v...


In [57]:
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-cmg-2.chn#data"
cost_history = []
data_ = scrape_table_data('QST', 'CTCP Sách và Thiết bị Trường học Quảng Ninh',url)
cost_history.extend(data_)  

Nhập thành công: QST - CTCP Sách và Thiết bị Trường học Quảng Ninh
Tổng số trang: 199
Đang lấy dữ liệu trang 179...
Lỗi khi lấy dữ liệu từ trang 179: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=133.0.6943.141); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x602159b5614a <unknown>
#1 0x6021595f3b80 <unknown>
#2 0x6021595fa913 <unknown>
#3 0x6021595fcea8 <unknown>
#4 0x6021595fcf33 <unknown>
#5 0x60215963f57d <unknown>
#6 0x60215966aed2 <unknown>
#7 0x602159638a2a <unknown>
#8 0x60215966b09e <unknown>
#9 0x60215969114a <unknown>
#10 0x60215966aca3 <unknown>
#11 0x602159636f08 <unknown>
#12 0x602159638071 <unknown>
#13 0x602159b1fb5b <unknown>
#14 0x602159b23ae2 <unknown>
#15 0x602159b0b967 <unknown>
#16 0x602159b246d4 <unknown>
#17 0x602159aefc7f <unknown>
#18 0x602159b44cd8 <unknown>
#19 0x602159b44ea9 <unkn

In [13]:
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-cmg-3.chn#data"

cost_history = []  # Danh sách để lưu toàn bộ dữ liệu

for stock_code, company_name in zip(df_['Mã CK'], df_['Tên công ty']):
    data_ = scrape_table_data(stock_code, company_name, url)
    cost_history.extend(data_)  # Thêm dữ liệu vào danh sách chính

Nhập thành công: STC - CTCP Sách và Thiết bị Trường học Thành phố Hồ Chí Minh
Tổng số trang: 226
Đang lấy dữ liệu trang 1...
Đang lấy dữ liệu trang 2...
Đang lấy dữ liệu trang 3...
Đang lấy dữ liệu trang 4...
Đang lấy dữ liệu trang 5...
Đang lấy dữ liệu trang 6...
Đang lấy dữ liệu trang 7...
Đang lấy dữ liệu trang 8...
Đang lấy dữ liệu trang 9...
Đang lấy dữ liệu trang 10...
Đang lấy dữ liệu trang 11...
Đang lấy dữ liệu trang 12...
Đang lấy dữ liệu trang 13...
Đang lấy dữ liệu trang 14...
Đang lấy dữ liệu trang 15...
Đang lấy dữ liệu trang 16...
Đang lấy dữ liệu trang 17...
Đang lấy dữ liệu trang 18...
Đang lấy dữ liệu trang 19...
Đang lấy dữ liệu trang 20...
Đang lấy dữ liệu trang 21...
Đang lấy dữ liệu trang 22...
Đang lấy dữ liệu trang 23...
Đang lấy dữ liệu trang 24...
Đang lấy dữ liệu trang 25...
Đang lấy dữ liệu trang 26...
Đang lấy dữ liệu trang 27...
Đang lấy dữ liệu trang 28...
Đang lấy dữ liệu trang 29...
Đang lấy dữ liệu trang 30...
Đang lấy dữ liệu trang 31...
Đang lấy dữ l

In [10]:
cost_history

[['CST',
  'CTCP Than Cao Sơn - TKV',
  'Ngày',
  'Giá (nghìn VNĐ)',
  'Thay đổi',
  'GD khớp lệnh',
  'GD thỏa thuận',
  'Giá (nghìn VNĐ)'],
 ['CST',
  'CTCP Than Cao Sơn - TKV',
  'Đóng cửa',
  'Điều chỉnh',
  'Khối lượng',
  'Giá trị (tỷ VNĐ)',
  'Khối lượng',
  'Giá trị (tỷ VNĐ)',
  'Mở cửa',
  'Cao nhất',
  'Thấp nhất'],
 ['CST',
  'CTCP Than Cao Sơn - TKV',
  '19/03/2025',
  '23.6',
  '--',
  '0(0.00 %)',
  '25,247',
  '0.59',
  '0',
  '0',
  '23.5',
  '23.6',
  '23.3'],
 ['CST',
  'CTCP Than Cao Sơn - TKV',
  '18/03/2025',
  '23.6',
  '--',
  '-0.1(-0.42 %)',
  '66,010',
  '1.57',
  '0',
  '0',
  '23.9',
  '24',
  '23.5'],
 ['CST',
  'CTCP Than Cao Sơn - TKV',
  '14/03/2025',
  '23.1',
  '--',
  '0.1(0.43 %)',
  '13,600',
  '0.31',
  '0',
  '0',
  '23',
  '23.1',
  '22.7'],
 ['CST',
  'CTCP Than Cao Sơn - TKV',
  '13/03/2025',
  '23',
  '--',
  '0.2(0.88 %)',
  '28,210',
  '0.64',
  '0',
  '0',
  '22.9',
  '23.1',
  '22.7'],
 ['CST',
  'CTCP Than Cao Sơn - TKV',
  '12/03/2025',


In [21]:
# Lưu vào CSV
columns = ["Stock_code", "Tên công ty","Ngày", "Giá đóng cửa", "Giá điều chỉnh", "Thay đổi", "Khối lượng khớp lệnh",
        "Giá trị khớp lệnh", "Khối lượng thỏa thuận", "Giá trị thỏa thuận", 
        "Giá mở cửa", "Giá cao nhất", "Giá thấp nhất"] #lịch sử giá
# columns = ['Stock_code', 'Tên công ty',
#     "Ngày", 
#     "Thay đổi", 
#     "Số lệnh Mua", 
#     "Khối lượng Mua", 
#     "KLTB 1 lệnh Mua", 
#     "Số lệnh Bán", 
#     "Khối lượng Bán", 
#     "KLTB 1 lệnh Bán", 
#     "Khối lượng ròng"
# ] #Thống kê đặt lệnh

# columns = ['Stock_code', 'Tên công ty',
#     "Ngày", "Thay đổi", "Giao dịch ròng - Khối lượng", "Giao dịch ròng - Giá trị (tỷ VNĐ)",
#     "Mua - Khối lượng", "Mua - Giá trị (tỷ VNĐ)", "Bán - Khối lượng", "Bán - Giá trị (tỷ VNĐ)",
#     "Room còn lại", "Đang sở hữu"
# ] #Khối ngoại

# columns = ['Stock_code', 'Tên công ty',
#     "Mã", "Ngày", "Khối lượng mua", "Giá trị mua(tỷ VND)", "Khối lượng bán", "Giá trị bán(tỷ VND)", "Giá trị ròng_Khối lượng", "Giá trị ròng_giá trị(Tỷ VND)"
# ] #Tự doanh


df_ = pd.DataFrame(cost_history, columns=columns)

df_.head()

,Stock_code,Tên công ty,Ngày,Giá đóng cửa,Giá điều chỉnh,Thay đổi,Khối lượng khớp lệnh,Giá trị khớp lệnh,Khối lượng thỏa thuận,Giá trị thỏa thuận,Giá mở cửa,Giá cao nhất,Giá thấp nhất
0,KDH,CTCP Đầu tư và Kinh doanh Nhà Khang Điền,Ngày,Giá (nghìn VNĐ),Thay đổi,GD khớp lệnh,GD thỏa thuận,Giá (nghìn VNĐ),None,None,None,None,None
1,KDH,CTCP Đầu tư và Kinh doanh Nhà Khang Điền,Đóng cửa,Điều chỉnh,Khối lượng,Giá trị (tỷ VNĐ),Khối lượng,Giá trị (tỷ VNĐ),Mở cửa,Cao nhất,Thấp nhất,None,None
2,KDH,CTCP Đầu tư và Kinh doanh Nhà Khang Điền,09/08/2019,23.6,15.04,0.2(0.85 %),"299,050",7.02,0,0,23.5,23.7,23.4
3,KDH,CTCP Đầu tư và Kinh doanh Nhà Khang Điền,08/08/2019,23.4,14.91,0.45(1.96 %),"432,680",10.1,0,0,22.95,23.5,22.95
4,KDH,CTCP Đầu tư và Kinh doanh Nhà Khang Điền,07/08/2019,22.95,14.63,0.45(2.00 %),"344,060",7.83,0,0,22.4,22.95,22.55


In [103]:
df['Mã CK'][9:]

9     DAD
10    DAE
11    DST
12    EBS
13    ECI
14    EID
15    EPH
16    FOC
17    FOX
18    GLT
19    HEV
20    ICT
21    KST
22    LBE
23    MFS
24    NBE
25    ODE
26    ONE
27    PMT
28    PTP
29    QST
30    SED
31    SGD
32    SGT
33    SMN
34    STC
35    TPH
36    TTN
37    VGI
38    VIE
39    VNB
40    VNX
41    VNZ
42    VTC
43    VTK
44    YEG
Name: Mã CK, dtype: object

ĐAng ở cái mã DAE. Mai chạy tiếp

In [106]:
df_.shape

(32415, 11)

In [12]:
df_.tail()

,Stock_code,Tên công ty,Ngày,Giá đóng cửa,Giá điều chỉnh,Thay đổi,Khối lượng khớp lệnh,Giá trị khớp lệnh,Khối lượng thỏa thuận,Giá trị thỏa thuận,Giá mở cửa,Giá cao nhất,Giá thấp nhất
58680,LPB,Ngân hàng TMCP Lộc Phát Việt Nam,11/10/2017,13.1,3.8,-0.7(-5.07 %),"1,929,895",25.69,"6,602",0.08,13.8,13.8,12.9
58681,LPB,Ngân hàng TMCP Lộc Phát Việt Nam,10/10/2017,13.7,3.97,-0.2(-1.44 %),"1,369,264",18.83,0,0,13.9,14,13.6
58682,LPB,Ngân hàng TMCP Lộc Phát Việt Nam,09/10/2017,13.9,4.03,0.1(0.72 %),"1,018,740",14.13,"371,030",4.98,13.8,14,13.7
58683,LPB,Ngân hàng TMCP Lộc Phát Việt Nam,06/10/2017,13.8,4,-0.4(-2.82 %),"1,196,846",16.49,"1,457,311",17.74,14.2,14.2,13.6
58684,LPB,Ngân hàng TMCP Lộc Phát Việt Nam,05/10/2017,14.2,4.11,-0.6(-4.05 %),"7,293,500",103.25,0,0,14.8,15.8,13.5


In [22]:
df_.to_csv(r"/home/hamornic/FinalXXX/DataScraped/lịch_sử_giá.csv", index=False, encoding="utf-8-sig", mode='a', header=False)

## News

In [55]:
def init_driver():
    """Khởi tạo và trả về một trình duyệt Chrome WebDriver."""
    chrome_options = ChromeOptions()
    # chrome_options.add_argument("--headless=new")  # Headless mode mới
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)


def scrape_company_news(stock_code, url):
    """Hàm lấy thông tin công ty từ URL và lưu vào danh sách all_results"""
    driver = init_driver()
    wait = WebDriverWait(driver, 10)
    news_list = []
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        
        driver.execute_script("window.scrollBy(0,200);")
        
        news_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="view-tab"]/div/div/li[6]/a')))
        news_button.click()
        print(f'🔍 Đang lấy news của stock: {url}')
        
        # Đợi bảng tin xuất hiện
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="newsevent-content"]/div/div/div[1]/div[2]/div[3]/table')))

        # Lấy tất cả các bảng tin trên trang
        table = driver.find_element(By.XPATH, '//*[@id="newsevent-content"]/div/div/div[1]/div[2]/div[3]/table')
        print('Đang lấy news')

        driver.execute_script("window.scrollBy(0, 800);")
            
        while True:
            # Lấy tất cả các tin tức trong bảng
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.text-link")))
            news_elements = table.find_elements(By.CSS_SELECTOR, "a.text-link")
            
            for news in news_elements:  
                title = news.text
                link = "https:" + news.get_attribute("href")  # Thêm "https:" vào link
                news_list.append((stock_code, title, link))

            # Kiểm tra nút "Next" có bị vô hiệu hóa không
            try:
                
                next_button = driver.find_element(By.XPATH, '//*[@id="newsevent-content"]/div/div/div[1]/div[2]/div[3]/table/tbody/tr[6]/td/div/div[2]')
                if "disabled" in next_button.get_attribute("outerHTML"):
                    break  # Nếu bị vô hiệu hóa thì dừng vòng lặp
                next_button.click()
                wait = WebDriverWait(driver, 10)
                time.sleep(3)  # Đợi trang tải lại
            except:
                break  # Nếu không tìm thấy nút "Next", thoát vòng lặp


    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ {url}: {e}")
    finally:
        driver.quit()
        
    return news_list

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   STT                 45 non-null     object
 1   Mã CK               45 non-null     object
 2   Tên công ty         45 non-null     object
 3   Ngành cấp 1         45 non-null     object
 4   Ngành cấp 2         45 non-null     object
 5   Ngành cấp 3         45 non-null     object
 6   Sàn                 45 non-null     object
 7   Ngày GDĐT           45 non-null     object
 8   Khối lượng NY/ĐKGD  45 non-null     object
 9   Link_detail         45 non-null     object
dtypes: object(10)
memory usage: 3.6+ KB


In [76]:
url = r'https://finance.vietstock.vn/BED-ctcp-sach-va-thiet-bi-truong-hoc-da-nang.htm'
data_ = scrape_company_news('BED', url)
data_   

🔍 Đang lấy news của stock: https://finance.vietstock.vn/BED-ctcp-sach-va-thiet-bi-truong-hoc-da-nang.htm
Đang lấy news


[('BED',
  'BED: Báo cáo tài chính năm 2024 (27/02/2025)',
  'https:https://vietstock.vn/2025/02/bed-bao-cao-tai-chinh-nam-2024-737-1275758.htm'),
 ('BED',
  'BED: Thay đổi giấy đăng ký kinh doanh (24/01/2025)',
  'https:https://vietstock.vn/2025/01/bed-thay-doi-giay-dang-ky-kinh-doanh-737-1266454.htm'),
 ('BED',
  'BED: Báo cáo quản trị công ty năm 2024 (24/01/2025)',
  'https:https://vietstock.vn/2025/01/bed-bao-cao-quan-tri-cong-ty-nam-2024-737-1266057.htm'),
 ('BED',
  'BED: Báo cáo tài chính quý 4/2024 (23/01/2025)',
  'https:https://vietstock.vn/2025/01/bed-bao-cao-tai-chinh-quy-4-2024-737-1265543.htm'),
 ('BED',
  'BED: Nghị quyết Hội đồng quản trị (29/11/2024)',
  'https:https://vietstock.vn/2024/11/bed-nghi-quyet-hoi-dong-quan-tri-737-1248908.htm'),
 ('BED',
  'BED: Báo cáo tài chính quý 3/2024 (18/10/2024)',
  'https:https://vietstock.vn/2024/10/bed-bao-cao-tai-chinh-quy-3-2024-737-1237463.htm'),
 ('BED',
  'BED: Cục quản lý thị trưởng TP Đà Nẵng Quyết định Xử phạt vi phạm hà

In [77]:
len(data_)

40

In [57]:
news_data = []

for stock_code, url in zip(df['Mã CK'], df['Link_detail']):
    data_ = scrape_company_news(stock_code, url)
    news_data.extend(data_) 

🔍 Đang lấy news của stock: https://finance.vietstock.vn/ABC-ctcp-truyen-thong-vmg.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/ABR-ctcp-dau-tu-nhan-hieu-viet.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/ADC-ctcp-my-thuat-va-truyen-thong.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/ADG-ctcp-clever-group.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/BDB-ctcp-sach-va-thiet-bi-binh-dinh.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/BED-ctcp-sach-va-thiet-bi-truong-hoc-da-nang.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/BST-ctcp-sach-va-thiet-bi-binh-thuan.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/CKV-ctcp-cokyvina.htm
Đang lấy news
🔍 Đang lấy news của stock: https://finance.vietstock.vn/CTR-tong-cong-ty-co-phan-cong-trinh-viettel.htm
Đang lấy news
🔍 Đang lấy news của stock: https://financ

In [58]:
len(news_data)

4474

In [59]:
news_data

[('ABC',
  'ABC: Ngày đăng ký cuối cùng Tham dự Đại hội đồng cổ đông thường niên năm 2025 (28/02/2025)',
  'https:https://vietstock.vn/2025/02/abc-ngay-dang-ky-cuoi-cung-tham-du-dai-hoi-dong-co-dong-thuong-nien-nam-2025-737-1276170.htm'),
 ('ABC',
  'BTG: Báo cáo thay đổi sở hữu cổ phiếu của cổ đông lớn - Công ty cổ phần đầu tư quốc tế ABC Group (28/02/2025)',
  'https:https://vietstock.vn/2025/02/btg-bao-cao-thay-doi-so-huu-co-phieu-cua-co-dong-lon-cong-ty-co-phan-dau-tu-quoc-te-abc-group-737-1276117.htm'),
 ('ABC',
  'ABC: Thông báo ngày đăng ký cuối cùng chốt danh sách cổ đông tham dự Đại hội đồng cổ đông thường niên năm 2025 (25/02/2025)',
  'https:https://vietstock.vn/2025/02/abc-thong-bao-ngay-dang-ky-cuoi-cung-chot-danh-sach-co-dong-tham-du-dai-hoi-dong-co-dong-thuong-nien-nam-2025-737-1274807.htm'),
 ('ABC',
  'ABC: Nghị quyết hội đồng quản trị (25/02/2025)',
  'https:https://vietstock.vn/2025/02/abc-nghi-quyet-hoi-dong-quan-tri-737-1274805.htm'),
 ('ABC',
  'ABC: Phán quyết củ

In [48]:
news_data[16]

('Truyền thông VMG trả cổ tức 5% bằng tiền (20/08/2024)',
 'https:https://vietstock.vn/2024/08/truyen-thong-vmg-tra-co-tuc-5-bang-tien-738-1221016.htm')

In [60]:
# Chuyển thành DataFrame
df = pd.DataFrame(news_data, columns=["stock_code", "news_content", "url"])

# Sửa lại cột URL (bỏ phần "https:https://")
df["url"] = df["url"].str.replace("https:https://", "https://", regex=False)

# Trích xuất ngày đăng tin từ `news_content`
df["uploaded_date"] = df["news_content"].str.extract(r'\((\d{2}/\d{2}/\d{4})\)')

# Loại bỏ ngày khỏi nội dung tin tức
df["news_content"] = df["news_content"].str.replace(r'\s*\(\d{2}/\d{2}/\d{4}\)$', '', regex=True)


In [61]:
df

,stock_code,news_content,url,uploaded_date
0,ABC,ABC: Ngày đăng ký cuối cùng Tham dự Đại hội đồ...,https://vietstock.vn/2025/02/abc-ngay-dang-ky-...,28/02/2025
1,ABC,BTG: Báo cáo thay đổi sở hữu cổ phiếu của cổ đ...,https://vietstock.vn/2025/02/btg-bao-cao-thay-...,28/02/2025
2,ABC,ABC: Thông báo ngày đăng ký cuối cùng chốt dan...,https://vietstock.vn/2025/02/abc-thong-bao-nga...,25/02/2025
3,ABC,ABC: Nghị quyết hội đồng quản trị,https://vietstock.vn/2025/02/abc-nghi-quyet-ho...,25/02/2025
4,ABC,ABC: Phán quyết của trọng tài quốc tế Singapor...,https://vietstock.vn/2025/02/abc-phan-quyet-cu...,24/02/2025
...,...,...,...,...
4469,YEG,YEG: Báo cáo kết quả giao dịch quyền mua cổ ph...,https://vietstock.vn/2025/03/yeg-bao-cao-ket-q...,04/03/2025
4470,YEG,YEG: Báo cáo kết quả giao dịch quyền mua cổ ph...,https://vietstock.vn/2025/03/yeg-bao-cao-ket-q...,03/03/2025
4471,YEG,YEG: Báo cáo kết quả giao dịch quyền mua cổ ph...,https://vietstock.vn/2025/03/yeg-bao-cao-ket-q...,03/03/2025
4472,YEG,Tổng Giám đốc YEG mua hơn 1.9 triệu cp,https://vietstock.vn/2025/03/tong-giam-doc-yeg...,01/03/2025


In [63]:
df_count = df.groupby(['stock_code'])['news_content'].count()

In [75]:
df_count[df_count < 50]

stock_code
BED    40
BST    30
CKV    25
EBS    40
LBE    45
NBE    20
ODE    20
ONE    20
SMN    25
TTN    35
VTC    25
VTK    15
YEG    15
Name: news_content, dtype: int64

Còn 1 số bị thiếu.

In [66]:
df.to_csv(r"./DataScraped/news/news.csv", index=False, encoding="utf-8-sig")

## Dividents and addition issuan

In [117]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def init_driver():
    """Khởi tạo và trả về một trình duyệt Chrome WebDriver."""
    chrome_options = ChromeOptions()
    chrome_options.add_argument("--headless=new")  # Headless mode nếu muốn chạy ẩn
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def scrape_table_dividends(ma_ck):
    """Lấy dữ liệu giao dịch từ trang web."""
    driver = init_driver()
    wait = WebDriverWait(driver, 10)
    data = []
    num_page = 1

    try:
        while True:
            driver.get(f'https://finance.vietstock.vn/lich-su-kien.htm?page={num_page}&code={ma_ck}')
            
            # Chờ dropdown xuất hiện rồi chọn giá trị 50
            try:
                dropdown = wait.until(EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="event-calendar-content"]/div/div[4]/div[1]/div/select')))
                select = Select(dropdown)
                select.select_by_value("50")
            except Exception:
                print("Không tìm thấy dropdown hoặc không thể chọn giá trị.")
                break

            # Chờ bảng xuất hiện
            try:
                wait.until(EC.presence_of_element_located((By.ID, "event-content")))
                table = driver.find_element(By.ID, "event-content")
            except:
                print(f"Không có dữ liệu cho mã {ma_ck} trên trang {num_page}. Dừng lấy dữ liệu.")
                break

            # Lấy danh sách hàng trong bảng
            rows = table.find_elements(By.XPATH, ".//tbody/tr")
            if not rows:
                print(f"Không có sự kiện cho mã {ma_ck}. Dừng lấy dữ liệu.")
                break

            print(f'Đang lấy dữ liệu của mã: {ma_ck}, trang {num_page}')
            
            # Lặp qua từng hàng để lấy dữ liệu
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 8:  # Đảm bảo có đủ số cột
                    data.append([
                        cells[0].text.strip(),  # STT
                        cells[1].text.strip(),  # Mã CK
                        cells[2].text.strip(),  # Sàn
                        cells[3].text.strip(),  # Ngày GDKHQ
                        cells[4].text.strip(),  # Ngày ĐKCC
                        cells[5].text.strip(),  # Ngày thực hiện
                        cells[6].text.strip(),  # Nội dung sự kiện
                        cells[7].text.strip(),  # Loại sự kiện
                    ])
            
            num_page += 1  # Chuyển sang trang tiếp theo
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ {ma_ck}: {e}")
    finally:
        driver.quit()  # Đảm bảo đóng trình duyệt sau khi hoàn thành

    return data


In [116]:
scrape_table_dividends('VTC')

Đang lấy dữ liệu của mã: VTC, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.


[['1',
  'VTC',
  'HNX',
  '26/03/2024',
  '27/03/2024',
  '24/04/2024',
  'Trả cổ tức năm 2022 bằng tiền, 700 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['2',
  'VTC',
  'HNX',
  '24/03/2023',
  '27/03/2023',
  '26/04/2023',
  'Trả cổ tức năm 2021 bằng tiền, 600 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['3',
  'VTC',
  'HNX',
  '25/03/2022',
  '28/03/2022',
  '27/04/2022',
  'Trả cổ tức năm 2020 bằng tiền, 1,200 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['4',
  'VTC',
  'HNX',
  '25/03/2021',
  '26/03/2021',
  '27/04/2021',
  'Trả cổ tức năm 2019 bằng tiền, 1,100 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['5',
  'VTC',
  'HNX',
  '26/03/2020',
  '27/03/2020',
  '15/04/2020',
  'Trả cổ tức năm 2018 bằng tiền, 1,000 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['6',
  'VTC',
  'HNX',
  '22/03/2019',
  '25/03/2019',
  '10/04/2019',
  'Trả cổ tức năm 2017 bằng tiền, 800 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['7',
  'VTC',
  'HNX',
  '15/03/2017',
  '16/03/2017',
  '03/04/2017',
  'Trả cổ

In [119]:
dividents_data = []
for stock_code in df['Mã CK']:
    data_ = scrape_table_dividends(stock_code)
    dividents_data.extend(data_)

Đang lấy dữ liệu của mã: ABC, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: ABR, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: ADC, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: ADG, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: BDB, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: BED, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: BST, trang 1
Đang lấy dữ liệu của mã: BST, trang 2
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: CKV, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: CTR, trang 1
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ liệu của mã: DAD, trang 1
Đang lấy dữ liệu của mã: DAD, trang 2
Không tìm thấy dropdown hoặc không thể chọn giá trị.
Đang lấy dữ li

In [121]:
dividents_data

[['1',
  'ABC',
  'UPCoM',
  '05/09/2024',
  '06/09/2024',
  '11/10/2024',
  'Trả cổ tức năm 2023 bằng tiền, 500 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['2',
  'ABC',
  'UPCoM',
  '31/08/2023',
  '05/09/2023',
  '29/09/2023',
  'Trả cổ tức năm 2022 bằng tiền, 1,500 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['3',
  'ABC',
  'UPCoM',
  '08/07/2020',
  '09/07/2020',
  '12/08/2020',
  'Trả cổ tức năm 2019 bằng tiền, 1,000 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['4',
  'ABC',
  'UPCoM',
  '25/06/2019',
  '26/06/2019',
  '10/07/2019',
  'Trả cổ tức năm 2018 bằng tiền, 1,000 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['5',
  'ABC',
  'UPCoM',
  '12/07/2017',
  '13/07/2017',
  '27/07/2017',
  'Trả cổ tức năm 2016 bằng tiền, 19,500 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['6',
  'ABC',
  'UPCoM',
  '26/05/2016',
  '27/05/2016',
  '24/06/2016',
  'Trả cổ tức năm 2015 bằng tiền, 4000 đồng/CP',
  'Trả cổ tức bằng tiền mặt'],
 ['7',
  'ABC',
  'UPCoM',
  '25/05/2015',
  '27/05/2015',
  '25/0

In [122]:
df = pd.DataFrame(dividents_data, columns=[
    "No.", "Stock", "Exchange", "Ex-Right Date▼", "Record Date", "Exercise Date", "Event content", "Event type"
])
df.head()

,No.,Stock,Exchange,Ex-Right Date▼,Record Date,Exercise Date,Event content,Event type
0,1,ABC,UPCoM,05/09/2024,06/09/2024,11/10/2024,"Trả cổ tức năm 2023 bằng tiền, 500 đồng/CP",Trả cổ tức bằng tiền mặt
1,2,ABC,UPCoM,31/08/2023,05/09/2023,29/09/2023,"Trả cổ tức năm 2022 bằng tiền, 1,500 đồng/CP",Trả cổ tức bằng tiền mặt
2,3,ABC,UPCoM,08/07/2020,09/07/2020,12/08/2020,"Trả cổ tức năm 2019 bằng tiền, 1,000 đồng/CP",Trả cổ tức bằng tiền mặt
3,4,ABC,UPCoM,25/06/2019,26/06/2019,10/07/2019,"Trả cổ tức năm 2018 bằng tiền, 1,000 đồng/CP",Trả cổ tức bằng tiền mặt
4,5,ABC,UPCoM,12/07/2017,13/07/2017,27/07/2017,"Trả cổ tức năm 2016 bằng tiền, 19,500 đồng/CP",Trả cổ tức bằng tiền mặt


In [123]:
df.to_csv(r"./DataScraped/Dividents/dividents.csv", index=False, encoding="utf-8-sig")

## Leadership and ownership

In [57]:
from collections import defaultdict
def init_driver():
    """Khởi tạo và trả về một trình duyệt Chrome WebDriver."""
    chrome_options = ChromeOptions()
    chrome_options.add_argument("--headless=new")  # Headless mode nếu muốn chạy ẩn
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # Không tải ảnh
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

def scrape_table_ship(ma_ck):
    """Lấy dữ liệu giao dịch từ trang web."""
    driver = init_driver()
    wait = WebDriverWait(driver, 10)
    
    ceo_data = defaultdict(list)   # Dữ liệu bảng 1 (chức vụ)
    shareholder_data = defaultdict(list)  # Dữ liệu bảng 2 (cổ đông)
    merged_data = {}


    try:
        driver.get(r'https://cafef.vn/')
        WebDriverWait(driver, 5)
        
        # Chờ và tìm ô nhập liệu
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="CafeF_SearchKeyword_Company"]'))
)
        # Nhập thông tin vào ô tìm kiếm
        search_box.send_keys(ma_ck)  
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "bt_search"))
        )
        search_button.click()
        print(f'Đang lấy dữ liệu của mã: {ma_ck}')

        element = driver.find_element(By.XPATH, '//*[@id="lsTab3CT"]')
        driver.execute_script("arguments[0].scrollIntoView();", element)
        
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="lsTab3CT"]'))
        )
        search_button.click()
        
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.cl_ceo")))

        # Lấy danh sách các hàng trừ hàng tiêu đề
        rows = driver.find_elements(By.CSS_SELECTOR, "table.cl_ceo tr")[1:]

        for row in rows:
            # print('--------------------')
            cols = row.find_elements(By.TAG_NAME, "td")
            chuc_vu = cols[0].text.strip()
            ten = cols[1].text.strip()
            ten_split = ten.split()
            ten_co_dong = ' '.join(ten_split[1:])
            ceo_data[ten_co_dong].append({"chuc_vu": chuc_vu})
            
        link_shares = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="divCoDongLon"]/a'))
        )
        link_shares.click()
        
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table[cellpadding='2']")))

        # Lấy danh sách hàng
        rows = driver.find_elements(By.CSS_SELECTOR, "table[cellpadding='2'] tr")[1:]  # Bỏ hàng tiêu đề

        # Lặp qua từng hàng và lấy thông tin    
        for row in rows:
            # print('----------------')
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 4:
                ten_co_dong = cols[0].text.strip()
                so_co_phieu = cols[1].text.strip()
                ty_le = cols[2].text.strip()
                tinh_den_ngay = cols[3].text.strip()
                shareholder_data[ten_co_dong].append({
                    "so_co_phieu": so_co_phieu, 
                    "ty_le": ty_le, 
                    "tinh_den_ngay": tinh_den_ngay
                })
                
        
        # Thêm dữ liệu từ CEO
        for ten, ceo_info in ceo_data.items():
            merged_data[ten] = {
                "chuc_vu": ceo_info[0]["chuc_vu"],
                "so_co_phieu": "",
                "ty_le": "",
                "tinh_den_ngay": ""
            }

        # Thêm dữ liệu từ cổ đông
        for ten, shareholder_info in shareholder_data.items():
            if ten in merged_data:
                merged_data[ten]["so_co_phieu"] = shareholder_info[0]["so_co_phieu"]
                merged_data[ten]["ty_le"] = shareholder_info[0]["ty_le"]
                merged_data[ten]["tinh_den_ngay"] = shareholder_info[0]["tinh_den_ngay"]
            else:
                merged_data[ten] = {
                    "chuc_vu": "",
                    "so_co_phieu": shareholder_info[0]["so_co_phieu"],
                    "ty_le": shareholder_info[0]["ty_le"],
                    "tinh_den_ngay": shareholder_info[0]["tinh_den_ngay"]
                }

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ {ma_ck}: {e}")
    finally:
        driver.quit()  # Đảm bảo đóng trình duyệt sau khi hoàn thành

    return merged_data

In [64]:
data_ = scrape_table_ship('ABC')

Đang lấy dữ liệu của mã: ABC


In [65]:
data_

{'tên': {'chuc_vu': 'Chức vụ',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Mạnh Hùng': {'chuc_vu': 'Chủ tịch HĐQT',
  'so_co_phieu': '276,770',
  'ty_le': '1.36',
  'tinh_den_ngay': '05/09/2024'},
 'Nguyễn Thanh Hải': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Võ Thăng Long': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Thị Thùy Dương': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Lương Minh Tuấn': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nghiêm Lê Hương Quỳnh': {'chuc_vu': 'Phụ trách quản trị',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Hoàng Nam': {'chuc_vu': 'Tổng Giám đốc',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Đức Thọ': {'chuc_vu': 'Phó Tổng Giám đốc',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den

In [70]:
data_

{'tên': {'chuc_vu': 'Chức vụ',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Mạnh Hùng': {'chuc_vu': 'Chủ tịch HĐQT',
  'so_co_phieu': '276,770',
  'ty_le': '1.36',
  'tinh_den_ngay': '05/09/2024'},
 'Nguyễn Thanh Hải': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Võ Thăng Long': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Thị Thùy Dương': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Lương Minh Tuấn': {'chuc_vu': 'Thành viên HĐQT',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nghiêm Lê Hương Quỳnh': {'chuc_vu': 'Phụ trách quản trị',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Hoàng Nam': {'chuc_vu': 'Tổng Giám đốc',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den_ngay': ''},
 'Nguyễn Đức Thọ': {'chuc_vu': 'Phó Tổng Giám đốc',
  'so_co_phieu': '',
  'ty_le': '',
  'tinh_den

In [71]:
lead_shares={}
lead_shares['ABC'] = data_

In [72]:
lead_shares

{'ABC': {'tên': {'chuc_vu': 'Chức vụ',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Mạnh Hùng': {'chuc_vu': 'Chủ tịch HĐQT',
   'so_co_phieu': '276,770',
   'ty_le': '1.36',
   'tinh_den_ngay': '05/09/2024'},
  'Nguyễn Thanh Hải': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Võ Thăng Long': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Thị Thùy Dương': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Lương Minh Tuấn': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nghiêm Lê Hương Quỳnh': {'chuc_vu': 'Phụ trách quản trị',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Hoàng Nam': {'chuc_vu': 'Tổng Giám đốc',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Đức Thọ': {'chuc_vu': 'Phó Tổng Giám đốc',
   'so_

In [75]:
lead_shares = {}

for stock_code in df['Mã CK']:
    data_ = scrape_table_ship(stock_code)
    lead_shares[stock_code] = data_

Đang lấy dữ liệu của mã: ABC
Đang lấy dữ liệu của mã: ABR
Đang lấy dữ liệu của mã: ADC
Đang lấy dữ liệu của mã: ADG
Đang lấy dữ liệu của mã: BDB
Đang lấy dữ liệu của mã: BED
Đang lấy dữ liệu của mã: BST
Đang lấy dữ liệu của mã: CKV
Đang lấy dữ liệu của mã: CTR
Đang lấy dữ liệu của mã: DAD
Đang lấy dữ liệu của mã: DAE
Đang lấy dữ liệu của mã: DST
Đang lấy dữ liệu của mã: EBS
Đang lấy dữ liệu của mã: ECI
Đang lấy dữ liệu của mã: EID
Đang lấy dữ liệu của mã: EPH
Đang lấy dữ liệu của mã: FOC
Đang lấy dữ liệu của mã: FOX
Đang lấy dữ liệu của mã: GLT
Đang lấy dữ liệu của mã: HEV
Đang lấy dữ liệu của mã: ICT
Đang lấy dữ liệu của mã: KST
Đang lấy dữ liệu của mã: LBE
Đang lấy dữ liệu của mã: MFS
Đang lấy dữ liệu của mã: NBE
Đang lấy dữ liệu của mã: ODE
Đang lấy dữ liệu của mã: ONE
Đang lấy dữ liệu của mã: PMT
Đang lấy dữ liệu của mã: PTP
Đang lấy dữ liệu của mã: QST
Đang lấy dữ liệu của mã: SED
Đang lấy dữ liệu của mã: SGD
Đang lấy dữ liệu của mã: SGT
Đang lấy dữ liệu của mã: SMN
Đang lấy dữ li

In [76]:
lead_shares

{'ABC': {'tên': {'chuc_vu': 'Chức vụ',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Mạnh Hùng': {'chuc_vu': 'Chủ tịch HĐQT',
   'so_co_phieu': '276,770',
   'ty_le': '1.36',
   'tinh_den_ngay': '05/09/2024'},
  'Nguyễn Thanh Hải': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Võ Thăng Long': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Thị Thùy Dương': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Lương Minh Tuấn': {'chuc_vu': 'Thành viên HĐQT',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nghiêm Lê Hương Quỳnh': {'chuc_vu': 'Phụ trách quản trị',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Hoàng Nam': {'chuc_vu': 'Tổng Giám đốc',
   'so_co_phieu': '',
   'ty_le': '',
   'tinh_den_ngay': ''},
  'Nguyễn Đức Thọ': {'chuc_vu': 'Phó Tổng Giám đốc',
   'so_

In [77]:
rows = []
for stock_code, people in lead_shares.items():
    for name, details in people.items():
        if name != "tên":  # Bỏ qua phần tiêu đề
            row = {"stock_code": stock_code, "name": name, **details}
            rows.append(row)

In [78]:
df = pd.DataFrame(rows)

In [79]:
df.head()

,stock_code,name,chuc_vu,so_co_phieu,ty_le,tinh_den_ngay
0,ABC,Nguyễn Mạnh Hùng,Chủ tịch HĐQT,"276,770",1.36,05/09/2024
1,ABC,Nguyễn Thanh Hải,Thành viên HĐQT,,,
2,ABC,Võ Thăng Long,Thành viên HĐQT,,,
3,ABC,Nguyễn Thị Thùy Dương,Thành viên HĐQT,,,
4,ABC,Lương Minh Tuấn,Thành viên HĐQT,,,


In [81]:
df.to_csv(r"/home/hamornic/FinalXXX/DataScraped/LeadOwnerShip/lead_shares.csv", index=False)